# Session 3, Module 2: Observability & Cost Optimization

---

## Module Overview

```
┌────────────────────────────────────────────────────────────────────┐
│                                                                    │
│  📚 Session 3, Module 2: Observability & Cost Optimization         │
│                                                                    │
│  🎯 Learning Objectives:                                           │
│     • Understand the observability landscape (LangSmithvs Langfuse)│
│     • Set up Langfuse for production tracing                       │
│     • Integrate observability into incident-postmortem project     │
│     • Learn cost optimization strategies (theory)                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [2]:
# Workshop Setup
import sys
sys.path.insert(0, '..')

from setup_llm import verify_setup, get_chat_model

verify_setup()

print("\n📚 Session 3, Module 2: Observability & Cost Optimization")
print("=" * 60)
print("\n🔗 Prerequisites:")
print("   • Module 1 complete (Evaluation Frameworks)")
print("   • OpenAI API key configured")
print("\n🎯 What you'll learn:")
print("   • When to use LangSmith vs Langfuse")
print("   • How to set up Langfuse for production tracing")
print("   • Cost optimization strategies for LLM applications")

🔍 Checking LLM Configuration...
📡 Provider: DIAL (Azure OpenAI via EPAM AI Proxy)
✅ DIAL_API_KEY is set

📋 Configuration:
   AZURE_OPENAI_ENDPOINT: https://ai-proxy.lab.epam.com
   AZURE_OPENAI_API_VERSION: 2024-08-01-preview
   AZURE_OPENAI_DEPLOYMENT_NAME: gpt-4

✅ DIAL setup verified successfully!

📚 Session 3, Module 2: Observability & Cost Optimization

🔗 Prerequisites:
   • Module 1 complete (Evaluation Frameworks)
   • OpenAI API key configured

🎯 What you'll learn:
   • When to use LangSmith vs Langfuse
   • How to set up Langfuse for production tracing
   • Cost optimization strategies for LLM applications


---

## Part 1: Why Observability Matters

### The Visibility Problem

LLM applications are inherently **black boxes**. Without observability:

| Problem | Impact |
|---------|--------|
| **No Debugging** | "It worked yesterday, why did it fail today?" |
| **No Cost Insight** | $10K bill surprise at month end |
| **No Performance Data** | "Is my agent getting slower?" |
| **No Quality Tracking** | "Are responses getting worse over time?" |

### What We Need to Observe

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    LLM OBSERVABILITY LAYERS                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ TRACE (End-to-end request)                                      │    │
│  │                                                                 │    │
│  │  ┌───────────────────────────────────────────────────────────┐  │    │
│  │  │ SPAN: Agent                                               │  │    │
│  │  │                                                           │  │    │
│  │  │  ┌────────────┐  ┌────────────┐  ┌────────────┐           │  │    │
│  │  │  │ GENERATION │  │ GENERATION │  │ GENERATION │           │  │    │
│  │  │  │ LLM Call 1 │  │ LLM Call 2 │  │ LLM Call 3 │           │  │    │
│  │  │  │            │  │            │  │            │           │  │    │
│  │  │  │ • Prompt   │  │ • Prompt   │  │ • Prompt   │           │  │    │
│  │  │  │ • Response │  │ • Response │  │ • Response │           │  │    │
│  │  │  │ • Tokens   │  │ • Tokens   │  │ • Tokens   │           │  │    │
│  │  │  │ • Latency  │  │ • Latency  │  │ • Latency  │           │  │    │
│  │  │  └────────────┘  └────────────┘  └────────────┘           │  │    │
│  │  │                                                           │  │    │
│  │  └───────────────────────────────────────────────────────────┘  │    │
│  │                                                                 │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                                                                         │
│  Key Metrics at Each Level:                                             │
│  • Trace: Total latency, total cost, success/failure                    │
│  • Span: Agent step duration, tool calls, state changes                 │
│  • Generation: Token count, model used, prompt/response content         │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Part 2: LangSmith vs Langfuse Comparison

**Lets take a look at available options:**

| Tool | Provider | Deployment | Best For | Open Source | Cost Tracking | Self-Hosted |
|------|----------|------------|----------|-------------|---------------|-------------|
| **LangSmith** | LangChain | Cloud only | LangChain/LangGraph users | ❌ | ⚠️ Limited | ❌ |
| **Langfuse** | Community | Cloud + Self-hosted | Framework-agnostic, privacy | ✅ | ✅ Built-in | ✅ |
| **Arize Phoenix** | Arize AI | Cloud + Self-hosted | ML teams, RAG analysis | ✅ | ✅ | ✅ |
| **Helicone** | Helicone | Cloud + Self-hosted | Cost optimization | ✅ | ✅ Excellent | ✅ |
| **OpenLLMetry** | Traceloop | Self-hosted | OpenTelemetry native, any backend | ✅ | ✅ | ✅ |
| **Lunary** | Lunary | Cloud + Self-hosted | OSS LangSmith alternative | ✅ | ✅ | ✅ |
| **OpenLIT** | OpenLIT | Self-hosted | OTel + GPU monitoring | ✅ | ✅ | ✅ |

### The Two Major Players

| Aspect | **LangSmith** | **Langfuse** |
|--------|---------------|--------------|
| **Provider** | LangChain (commercial) | Open-source community |
| **Best For** | LangChain/LangGraph native | Any LLM framework, privacy-focused |
| **Deployment** | Cloud-only | Cloud OR self-hosted |
| **Pricing** | Free tier → paid | Free OSS, paid cloud |
| **Integration** | Automatic w/ LangChain | Manual setup (easy) |
| **Unique Features** | Prompt hub, native datasets | Cost tracking, public analytics |

### When to Choose Which?

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    DECISION TREE                                        │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Are you using LangChain/LangGraph?                                     │
│      │                                                                  │
│      ├─ YES → Do you need self-hosting or cost tracking?                │
│      │            │                                                     │
│      │            ├─ YES → Langfuse ✓                                   │
│      │            │                                                     │
│      │            └─ NO → LangSmith ✓ (tightest integration)            │
│      │                                                                  │
│      └─ NO → Do you need open-source/self-hosted?                       │
│                   │                                                     │
│                   ├─ YES → Langfuse ✓                                   │
│                   │                                                     │
│                   └─ NO → Either works, Langfuse has broader support    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Feature Comparison Table

| Feature | LangSmith | Langfuse |
|---------|-----------|----------|
| **Tracing** | ✅ Automatic (LangChain) | ✅ Callback handler |
| **Cost Tracking** | ❌ Limited | ✅ Built-in |
| **Self-Hosted** | ❌ No | ✅ Docker/K8s |
| **Prompt Management** | ✅ Prompt Hub | ✅ Prompt versioning |
| **Evaluation** | ✅ Native | ✅ External (DeepEval, etc.) |
| **User Feedback** | ✅ Thumbs up/down | ✅ Custom scores |
| **Session Tracking** | ✅ Thread-based | ✅ Session IDs |
| **Multi-model** | ✅ Any (via LangChain) | ✅ Any model |
| **SOC2/HIPAA** | ✅ Enterprise | ✅ Self-host for compliance |

### Our Choice: Langfuse

For this workshop, we'll use **Langfuse** because:

1. **Open-source** - No vendor lock-in, can self-host
2. **Cost tracking** - Built-in token cost calculation
3. **Framework agnostic** - Works with any LLM setup
4. **Privacy-friendly** - Self-host for sensitive data
5. **LangGraph compatible** - Easy CallbackHandler integration

---

## Part 3: Langfuse Setup & Demo

### Getting Started with Langfuse

#### Option 1: Langfuse Cloud (Recommended for Workshop)

1. Sign up at [langfuse.com](https://cloud.langfuse.com)
2. Create a new project
3. Copy your API keys

#### Option 2: Self-Hosted (Production)

```bash
# Docker Compose
git clone https://github.com/langfuse/langfuse.git
cd langfuse
docker-compose up -d
```

In [3]:
# Install Langfuse
# !pip install langfuse langchain-openai

print("Langfuse Installation & Setup")
print("=" * 60)
print("""
To install Langfuse:

    pip install langfuse langchain-openai

Set environment variables:

    export LANGFUSE_SECRET_KEY="sk-lf-..."
    export LANGFUSE_PUBLIC_KEY="pk-lf-..."
    export LANGFUSE_HOST="https://cloud.langfuse.com"  # or self-hosted URL

Or set in Python:

    import os
    os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
    os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
""")

Langfuse Installation & Setup

To install Langfuse:

    pip install langfuse langchain-openai

Set environment variables:

    export LANGFUSE_SECRET_KEY="sk-lf-..."
    export LANGFUSE_PUBLIC_KEY="pk-lf-..."
    export LANGFUSE_HOST="https://cloud.langfuse.com"  # or self-hosted URL

Or set in Python:

    import os
    os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
    os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."



# Basic Langfuse Integration with LangChain/LangGraph

```python

from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI

# Create Langfuse callback handler
langfuse_handler = CallbackHandler()

# Use with LangChain
llm = ChatOpenAI(model="gpt-4o")
response = llm.invoke(
    "What is the capital of France?",
    config={"callbacks": [langfuse_handler]}
)

# Traces automatically appear in Langfuse dashboard!


```

# Langfuse with LangGraph Integration

```python
from langfuse.callback import CallbackHandler
from langgraph.graph import StateGraph

# Create handler with trace metadata
langfuse_handler = CallbackHandler(
    user_id="user-123",           # Track by user
    session_id="session-abc",     # Group related traces
    tags=["production", "v1.2"]   # Custom tags for filtering
)

# Pass to LangGraph invoke
graph = create_postmortem_graph()  # Your LangGraph workflow
result = graph.invoke(
    initial_state,
    config={
        "callbacks": [langfuse_handler],
        "run_name": "incident-postmortem"  # Name in dashboard
    }
)

# All nodes, LLM calls, and tool invocations are traced!
```

# Integrating Langfuse with Incident-PostMortem Project

```python
"""Main entry point with Langfuse observability."""

import os
from langfuse.callback import CallbackHandler
from graph.workflow import create_postmortem_graph
from graph.state import IncidentState


def create_langfuse_handler(incident_id: str, severity: str) -> CallbackHandler:
    """Create a configured Langfuse handler for incident analysis."""
    return CallbackHandler(
        session_id=f"incident-{incident_id}",
        user_id=os.getenv("ONCALL_USER", "system"),
        tags=[
            f"severity:{severity}",
            "incident-postmortem",
            os.getenv("ENVIRONMENT", "development")
        ],
        metadata={
            "incident_id": incident_id,
            "severity": severity,
        }
    )


def analyze_incident(incident_data: dict) -> dict:
    """Run the incident postmortem workflow with tracing."""
    
    # Create Langfuse handler
    langfuse_handler = create_langfuse_handler(
        incident_id=incident_data.get("incident_id", "unknown"),
        severity=incident_data.get("severity", "unknown")
    )
    
    # Create graph
    graph = create_postmortem_graph()
    
    # Initialize state
    initial_state = IncidentState(
        incident_id=incident_data["incident_id"],
        severity=incident_data["severity"],
        logs=incident_data["logs"],
        timeline=[]
    )
    
    # Run with Langfuse tracing
    result = graph.invoke(
        initial_state,
        config={
            "callbacks": [langfuse_handler],
            "run_name": f"postmortem-{incident_data['incident_id']}"
        }
    )
    
    # Flush traces before returning
    langfuse_handler.flush()
    
    return result


if __name__ == "__main__":
    # Example usage
    incident = {
        "incident_id": "INC-2024-001",
        "severity": "SEV2",
        "logs": "... log data ..."
    }
    result = analyze_incident(incident)
```

# Advanced: Custom Spans with @observe Decorator

```python
from langfuse.decorators import observe, langfuse_context

@observe()
def analyze_logs(logs: str) -> dict:
    """Custom function with automatic tracing."""
    
    # Add custom metadata to the span
    langfuse_context.update_current_observation(
        metadata={"log_lines": len(logs.split("\\n"))},
        tags=["log-analysis"]
    )
    
    # Your analysis logic here
    patterns = extract_patterns(logs)
    
    # Score the span (for evaluation)
    langfuse_context.score_current_observation(
        name="pattern_coverage",
        value=len(patterns) / 10,  # Normalized score
        comment="Number of patterns found"
    )
    
    return {"patterns": patterns}


@observe(name="root-cause-analysis")
def identify_root_cause(patterns: list, context: str) -> str:
    """Root cause analysis with custom span name."""
    
    # Nested function calls create child spans
    root_cause = llm_analyze(patterns, context)
    
    return root_cause
```

### What You'll See in Langfuse Dashboard

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     LANGFUSE TRACE VIEW                                 │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Trace: postmortem-INC-2024-001                                         │
│  ├── Session: incident-INC-2024-001                                     │
│  ├── User: oncall@company.com                                           │
│  ├── Tags: severity:SEV2, incident-postmortem, production               │
│  │                                                                      │
│  │  Timeline:                                                           │
│  │  ─────────────────────────────────────────────────────────────────   │
│  │                                                                      │
│  │  [0ms]    ┌─ log_analyzer_node ──────────────────┐                   │
│  │           │  ├─ ChatOpenAI (gpt-4o)              │                   │
│  │           │  │  Input: "Analyze these logs..."   │                   │
│  │           │  │  Output: "Pattern: DB timeout..." │                   │
│  │           │  │  Tokens: 450 in, 230 out          │                   │
│  │           │  │  Cost: $0.0068                    │                   │
│  │  [2.1s]   └──────────────────────────────────────┘                   │
│  │                                                                      │
│  │  [2.1s]   ┌─ root_cause_node ────────────────────┐                   │
│  │           │  ├─ ChatOpenAI (gpt-4o)              │                   │
│  │           │  │  Tokens: 380 in, 180 out          │                   │
│  │           │  │  Cost: $0.0054                    │                   │
│  │  [3.8s]   └──────────────────────────────────────┘                   │
│  │                                                                      │
│  │  [3.8s]   ┌─ writer_node ────────────────────────┐                   │
│  │           │  Cost: $0.0089                       │                   │
│  │  [5.2s]   └──────────────────────────────────────┘                   │
│  │                                                                      │
│  │  [5.2s]   ┌─ reviewer_node ──────────────────────┐                   │
│  │           │  Result: REVISION_NEEDED             │                   │
│  │  [6.8s]   └──────────────────────────────────────┘                   │
│  │                                                                      │
│  │  [6.8s]   ┌─ writer_node (retry) ────────────────┐                   │
│  │           │  Cost: $0.0102                       │                   │
│  │  [8.5s]   └──────────────────────────────────────┘                   │
│  │                                                                      │
│  │  Total: 8.5s | Cost: $0.0423 | Tokens: 2,340                         │
│  │                                                                      │
│  └────────────────────────────────────────────────────────────────────-─┘
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Part 4: Cost Optimization Strategies

### The Cost Challenge: Why This Matters

LLM costs can spiral quickly, especially with agentic workflows. Let's do the math:

**Example: Incident Postmortem System**
```
Single postmortem run:
├── Log Analyzer:     ~800 input + 400 output tokens
├── Root Cause:       ~600 input + 300 output tokens  
├── Writer:           ~1000 input + 1500 output tokens
├── Reviewer:         ~1800 input + 400 output tokens
├── Writer (retry):   ~2200 input + 1600 output tokens  ← Self-reflection loop!
└── Total:            ~6,400 input + 4,200 output tokens

With GPT-4o ($2.50/1M input, $10/1M output):
├── Input cost:  6,400 × $2.50/1M = $0.016
├── Output cost: 4,200 × $10/1M  = $0.042
└── Total per run: ~$0.06

At scale:
├── 100 incidents/day = $6/day
├── 1000 incidents/day = $60/day = $1,800/month
└── With 3 retries avg = $5,400/month 😱
```

| Factor | Impact | Real Example |
|--------|--------|--------------|
| **Multi-step reasoning** | Each agent step = LLM call | 5 nodes × $0.01 = $0.05/request |
| **Self-reflection loops** | Writer ↔ Reviewer iterates | 3 iterations = 3× cost |
| **Tool retry logic** | Failed tools get retried | Network errors → 2× calls |
| **Long context** | Full conversation history | 10K tokens → $0.025 input alone |
| **Embedding costs** | RAG retrieval | $0.0001/1K tokens (often overlooked) |

### 2025 LLM Pricing Reference

## OpenAI Models

| Model             | Input (per 1M tokens) | Output (per 1M tokens) | Best For |
|-------------------|------------------------|--------------------------|----------|
| **GPT-4o**        | $2.50                  | $10.00                   | Complex reasoning, multimodal tasks |
| **GPT-4o-mini**   | $0.15                  | $0.60                    | Lightweight tasks, classification |
| **GPT-4.1 mini**  | $0.40                  | $1.60                    | Lower-cost balanced model |
| **GPT-4.1**       | $2.00                  | $8.00                    | Balanced long-context model |

---

## Anthropic Claude Models

| Model                 | Input (per 1M tokens) | Output (per 1M tokens) | Best For |
|-----------------------|------------------------|--------------------------|----------|
| **Claude Haiku 4.5**  | $1.00                  | $5.00                    | Fast, cost-efficient workloads |
| **Claude Sonnet 4.5** | $3.00                  | $15.00                   | Balanced reasoning + coding |
| **Claude Opus 4**     | $15.00                 | $75.00                   | Deep reasoning, premium workloads |

---

## Cost Comparison (1M Input + 1M Output Tokens)

GPT-4o-mini: $0.75
Claude Haiku 4.5: $6.00
GPT-4o: $12.50
Claude Sonnet 4.5: $18.00
Claude Opus 4: $90.00

**Note:** Based on publicly available pricing from OpenAI and Anthropic (2025).


### Cost Optimization Hierarchy

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     COST OPTIMIZATION PYRAMID                          │
│                                                                         │
│  Start from the BOTTOM for biggest impact!                             │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│                            ┌─────┐                                      │
│                           /│ 10% │\    4. Model Selection               │
│                          / │     │ \   (gpt-4o → gpt-4o-mini)          │
│                         /  └─────┘  \  Easy but limited impact         │
│                        /      ▲      \                                  │
│                       ┌───────┴───────┐                                │
│                      /│      20%      │\   3. Prompt Engineering        │
│                     / │               │ \  (shorter, focused prompts)  │
│                    /  └───────────────┘  \                              │
│                   /          ▲            \                             │
│                  ┌───────────┴─────────────┐                           │
│                 /│          30%            │\  2. Caching               │
│                / │                         │ \ (semantic + exact)       │
│               /  └─────────────────────────┘  \                         │
│              /              ▲                  \                        │
│             ┌───────────────┴───────────────────┐                      │
│            │               40%                   │  1. Architecture     │
│            │                                     │  (fewer LLM calls)   │
│            └─────────────────────────────────────┘                      │
│                                                                         │
│  ⚡ Pro tip: Combining all strategies can reduce costs by 60-80%       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Strategy 1: Architecture Optimization (40% savings)

**The Problem**: Developers often use LLMs for tasks that don't need them.

**Before** - Every step uses an LLM:
```
User Query → LLM (intent) → LLM (extract) → LLM (plan) → LLM (execute) → LLM (format)
             ↑               ↑               ↑            ↑               ↑
           $0.01           $0.01           $0.02        $0.03           $0.01
           
Total: 5 LLM calls = $0.08 per query
```

**After** - Hybrid approach:
```
User Query → Regex/Rules → LLM (plan + execute) → Template
             ↑              ↑                      ↑
           FREE           $0.03                  FREE
           
Total: 1 LLM call = $0.03 per query (62% savings!)
```

#### Practical Examples

| Step | LLM Approach | Optimized Approach | Savings |
|------|--------------|-------------------|---------|
| **Intent Detection** | "Classify this query..." | Keyword matching + rules | 100% |
| **Entity Extraction** | "Extract the date from..." | Regex patterns | 100% |
| **Data Validation** | "Is this JSON valid?" | JSON schema validator | 100% |
| **Response Formatting** | "Format this as markdown..." | Jinja2 templates | 100% |
| **Simple Routing** | "Which agent handles this?" | Decision tree | 100% |

#### Real-World Pattern: Tiered Processing

```python
def process_query(query: str) -> str:
    # Tier 1: Rule-based (FREE)
    if is_greeting(query):
        return random.choice(GREETING_RESPONSES)
    
    if is_faq(query):
        return FAQ_DATABASE.get(query)
    
    # Tier 2: Simple LLM (CHEAP - gpt-4o-mini)
    if is_simple_question(query):
        return llm_mini.invoke(query)
    
    # Tier 3: Complex LLM (EXPENSIVE - gpt-4o) 
    return llm_full.invoke(query)
```

**Impact on Incident Postmortem**:
- Log parsing → Regex for timestamps/error codes (save $0.01)
- Severity detection → Keyword rules (save $0.01)
- Report templating → Jinja2 instead of LLM (save $0.02)
- **Estimated savings: 35% per run**

### Strategy 2: Caching (30% savings)

Caching is your **best friend** for repeated or similar queries.

- Anthropic: https://platform.claude.com/docs/en/build-with-claude/prompt-caching
- OpenAI: https://platform.openai.com/docs/guides/prompt-caching

#### Types of LLM Caching

| Cache Type | How It Works | Hit Rate | Best For |
|------------|--------------|----------|----------|
| **Exact Match** | Hash prompt → lookup | High (90%+) for FAQs | Repeated identical queries |
| **Semantic Cache** | Embed prompt → similarity search | Medium (60-80%) | Similar questions |
| **Prompt Prefix Cache** | Cache long system prompts | Very High (95%+) | Same instructions |
| **Response Cache** | Cache LLM outputs | High | Deterministic outputs |

#### OpenAI's Automatic Prompt Caching

OpenAI automatically caches prompt prefixes for **50% discount**:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    OPENAI PROMPT CACHING                                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  System Prompt (1000 tokens):                                           │
│  ┌─────────────────────────────────────────────────────────────────┐    │
│  │ "You are an expert incident analyst. Your job is to analyze     │    │
│  │  logs, identify patterns, determine root cause, and write       │    │
│  │  comprehensive postmortem reports following our template..."    │    │
│  └─────────────────────────────────────────────────────────────────┘    │
│                                                                         │
│  First Call:    1000 tokens × $2.50/1M = $0.0025 (full price)           │
│  Second Call:   1000 tokens × $1.25/1M = $0.00125 (50% cached!)         │
│  Third Call:    1000 tokens × $1.25/1M = $0.00125 (50% cached!)         │
│                                                                         │
│  Requirements:                                                          │
│  • Prompts must be >1024 tokens                                         │
│  • Identical prefix (character-by-character)                            │
│  • Cache expires after 5-10 minutes of inactivity                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

#### Caching Strategy by Use Case

| Use Case | Cache Strategy | Expected Savings |
|----------|---------------|------------------|
| **FAQ Bot** | Exact match + semantic | 70-90% |
| **Code Review** | Semantic (similar code patterns) | 40-60% |
| **Incident Analysis** | Prompt prefix (system prompt) | 20-30% |
| **Report Generation** | Limited (unique inputs) | 10-20% |

### Strategy 3: Prompt Engineering for Cost (20% savings)

Every token counts! Optimizing prompts reduces both input AND output costs.

#### Token Reduction Techniques

| Technique | Before | After | Token Savings |
|-----------|--------|-------|---------------|
| **Remove fluff** | "Please kindly provide a detailed summary of the following text, making sure to include all important points..." | "Summarize:" | ~80% |
| **Use abbreviations** | "incident_identification_number" | "inc_id" | ~60% |
| **JSON over prose** | "The error type is connection timeout and it occurred at 14:30" | `{"type":"timeout","time":"14:30"}` | ~40% |
| **Structured output** | Free-form explanation | JSON schema response | ~30% |

#### Real Example: Postmortem System Prompt

**Before** (847 tokens):
```
You are an expert incident management analyst with years of experience in 
identifying root causes of system failures. Your task is to carefully analyze 
the provided incident logs and produce a comprehensive postmortem report. 

Please make sure to include the following sections in your report:
1. Executive Summary - A brief overview of the incident
2. Timeline - A chronological sequence of events
3. Root Cause Analysis - Detailed investigation of what went wrong
4. Impact Assessment - How the incident affected users and systems
5. Action Items - Specific steps to prevent future occurrences

Remember to maintain a blameless tone throughout your analysis...
```

**After** (312 tokens - 63% reduction!):
```
Role: Incident analyst
Task: Analyze logs, write postmortem

Output JSON:
{
  "summary": "1-2 sentences",
  "timeline": [{"time": "HH:MM", "event": "..."}],
  "root_cause": "primary cause",
  "impact": {"users": N, "duration": "Xh"},
  "actions": [{"priority": "P0-P2", "action": "...", "owner": "..."}]
}

Tone: Blameless, factual
```

#### Output Length Control

```python
# Technique 1: Max tokens parameter
response = llm.invoke(
    prompt,
    max_tokens=500  # Hard limit on output
)

# Technique 2: Explicit length instructions
prompt = """
Summarize in EXACTLY 3 bullet points (max 20 words each):
{content}
"""

# Technique 3: Structured output with schema
from pydantic import BaseModel, Field

class Summary(BaseModel):
    main_point: str = Field(max_length=100)
    details: list[str] = Field(max_items=3)
```

#### The 80/20 Rule for Prompts

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    PROMPT TOKEN DISTRIBUTION                           │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Typical prompt breakdown:                                             │
│                                                                         │
│  ████████████████████████████░░░░░░░░░░  System instructions (60%)     │
│  ████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  Few-shot examples (20%)       │
│  ████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  Actual user input (10%)       │
│  ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  Output formatting (10%)       │
│                                                                         │
│  Optimization priority:                                                 │
│  1. System instructions → Move to cached prefix                        │
│  2. Few-shot examples → Reduce to 1-2, or use fine-tuning              │
│  3. Output formatting → Use JSON schema instead of descriptions         │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Strategy 4: Model Selection & Routing (10-50% savings)

**The key insight**: Not all tasks need GPT-4o!

#### Model Capability vs Cost Matrix

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MODEL SELECTION GUIDE                                │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Task Complexity                                                        │
│       ▲                                                                 │
│       │                        ┌─────────────┐                          │
│  High │                        │   GPT-4o    │  Complex reasoning       │
│       │                        │   Claude    │  Multi-step analysis     │
│       │                        │   Opus      │  Novel problems          │
│       │         ┌──────────────┴─────────────┘                          │
│       │         │      GPT-4o-mini                                      │
│       │         │      Claude Sonnet        │  Structured tasks         │
│  Med  │         │      Claude Haiku         │  Standard Q&A             │
│       │         │                           │  Code completion          │
│       │  ┌──────┴───────────────────────────┘                           │
│       │  │   Rules / Templates / Regex                                  │
│  Low  │  │   No LLM needed!                  │  Classification          │
│       │  │                                   │  Formatting              │
│       └──┴───────────────────────────────────┴──────────────────────▶   │
│          Low              Medium              High                      │
│                         Cost →                                          │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

#### Implementing a Model Router

```python
from enum import Enum
from typing import Callable

class TaskComplexity(Enum):
    SIMPLE = "simple"      # Classification, extraction
    MEDIUM = "medium"      # Summarization, Q&A
    COMPLEX = "complex"    # Reasoning, analysis, creativity

class ModelRouter:
    """Route tasks to appropriate models based on complexity."""
    
    MODEL_MAP = {
        TaskComplexity.SIMPLE: "gpt-4o-mini",   # $0.15/1M in
        TaskComplexity.MEDIUM: "gpt-4o-mini",   # $0.15/1M in
        TaskComplexity.COMPLEX: "gpt-4o",       # $2.50/1M in
    }
    
    COST_MAP = {
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        "gpt-4o": {"input": 2.50, "output": 10.00},
    }
    
    @classmethod
    def classify_task(cls, task: str) -> TaskComplexity:
        """Classify task complexity using heuristics."""
        
        # Simple tasks: Classification, yes/no, extraction
        simple_keywords = ["classify", "extract", "is this", "yes or no", "true or false"]
        if any(kw in task.lower() for kw in simple_keywords):
            return TaskComplexity.SIMPLE
        
        # Complex tasks: Analysis, reasoning, multi-step
        complex_keywords = ["analyze", "why", "explain", "compare", "reason", "strategy"]
        if any(kw in task.lower() for kw in complex_keywords):
            return TaskComplexity.COMPLEX
        
        return TaskComplexity.MEDIUM
    
    @classmethod
    def get_model(cls, task: str) -> str:
        complexity = cls.classify_task(task)
        return cls.MODEL_MAP[complexity]


# Usage in Incident Postmortem
class OptimizedPostmortemAgents:
    
    def log_analyzer(self, logs: str) -> dict:
        # Simple extraction task → cheap model
        model = ModelRouter.get_model("extract patterns from logs")
        return ChatOpenAI(model=model).invoke(...)
    
    def root_cause_analyzer(self, patterns: list) -> str:
        # Complex reasoning → expensive model
        model = ModelRouter.get_model("analyze why the system failed")
        return ChatOpenAI(model=model).invoke(...)
    
    def reviewer(self, report: str) -> dict:
        # Medium task (scoring) → cheap model
        model = ModelRouter.get_model("score this report quality")
        return ChatOpenAI(model=model).invoke(...)
```

#### Cost Impact of Model Routing

| Agent | Before (all GPT-4o) | After (routed) | Savings |
|-------|---------------------|----------------|---------|
| Log Analyzer | $0.015 | $0.001 (mini) | 93% |
| Root Cause | $0.020 | $0.020 (4o) | 0% |
| Writer | $0.025 | $0.025 (4o) | 0% |
| Reviewer | $0.015 | $0.001 (mini) | 93% |
| **Total** | **$0.075** | **$0.047** | **37%** |

### Strategy 5: Batch Processing & Async (OpenAI Batch API)

OpenAI offers **50% discount** for batch requests processed within 24 hours.

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    OPENAI BATCH API                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Standard API:                                                          │
│  Request → Process → Response (immediate, full price)                   │
│                                                                         │
│  Batch API:                                                             │
│  Request 1 ─┐                                                           │
│  Request 2 ─┼─→ Queue → Process within 24h → Results (50% off!)         │
│  Request 3 ─┘                                                           │
│                                                                         │
│  Perfect for:                                                           │
│  • Nightly report generation                                            │
│  • Bulk data processing                                                 │
│  • Non-urgent analysis                                                  │
│  • Evaluation/testing pipelines                                         │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

#### Batch API Example

```python
import openai
import json

# Prepare batch requests
requests = [
    {
        "custom_id": f"incident-{i}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "messages": [
                {"role": "system", "content": "Analyze this incident log..."},
                {"role": "user", "content": log_data}
            ]
        }
    }
    for i, log_data in enumerate(all_incidents)
]

# Write to JSONL file
with open("batch_input.jsonl", "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

# Upload and create batch
batch_file = openai.files.create(file=open("batch_input.jsonl", "rb"), purpose="batch")
batch = openai.batches.create(input_file_id=batch_file.id, endpoint="/v1/chat/completions")

# Check status later
status = openai.batches.retrieve(batch.id)
# status.status: "validating" → "in_progress" → "completed"
```

#### When to Use Batch API

| Scenario | Use Batch? | Reason |
|----------|------------|--------|
| Real-time chat | ❌ No | Users expect immediate response |
| Incident postmortem | ❌ No | On-call needs results now |
| Nightly report generation | ✅ Yes | Can wait until morning |
| Bulk evaluation runs | ✅ Yes | Running 1000s of test cases |
| Historical analysis | ✅ Yes | Processing old incidents |
| Training data generation | ✅ Yes | Creating synthetic datasets |

### Cost Monitoring with Langfuse

Langfuse automatically tracks costs when you use their callback handler. This is **essential** for:
- Budget tracking & alerts
- Cost attribution by feature/user
- ROI analysis per use case

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     LANGFUSE COST DASHBOARD                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  📊 Daily Cost Breakdown                     📅 Dec 9, 2024            │
│  ═══════════════════════════════════════════════════════════════════   │
│                                                                         │
│  Today: $45.23 (↑12% vs yesterday)                                     │
│                                                                         │
│  By Model:                                                              │
│  ├── gpt-4o:        $32.15 (71%)  ████████████████████░░░░░░░░░░       │
│  ├── gpt-4o-mini:   $8.45 (19%)   █████░░░░░░░░░░░░░░░░░░░░░░░░░       │
│  └── embeddings:    $4.63 (10%)   ███░░░░░░░░░░░░░░░░░░░░░░░░░░░       │
│                                                                         │
│  By Feature (using tags):                                               │
│  ├── incident-postmortem: $18.50  ██████████░░░░░░░░░░░░░░░░░░░        │
│  ├── chatbot:             $15.30  ████████░░░░░░░░░░░░░░░░░░░░░        │
│  └── code-review:         $11.43  ██████░░░░░░░░░░░░░░░░░░░░░░░        │
│                                                                         │
│  📈 Cost Trends (7 days):                                               │
│       $50 ┤        ╭─╮                                                  │
│           │      ╭─╯ ╰──╮                                               │
│       $40 ┤   ╭──╯      ╰─╮                                             │
│           │ ╭─╯           │                                             │
│       $30 ┼─╯             ╰──                                           │
│           Mon  Tue  Wed  Thu  Fri  Sat  Sun                             │
│                                                                         │
│  ⚠️  ALERT: Cost up 45% from yesterday                                 │
│  └── Root cause: 3 SEV1 incidents triggered extended analysis          │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

#### Setting Up Cost Alerts

```python
# In your observability.py or monitoring setup
def check_daily_costs():
    """Alert if daily costs exceed threshold."""
    from langfuse import Langfuse
    
    langfuse = Langfuse()
    
    # Get today's traces
    traces = langfuse.fetch_traces(
        from_timestamp=datetime.now().replace(hour=0, minute=0),
        to_timestamp=datetime.now()
    )
    
    total_cost = sum(t.total_cost or 0 for t in traces.data)
    
    if total_cost > DAILY_BUDGET:
        send_slack_alert(f"🚨 Daily LLM cost ${total_cost:.2f} exceeds budget ${DAILY_BUDGET}")
        
    return total_cost
```

### Complete Cost Optimization Checklist

Use this checklist when building or optimizing LLM applications:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                  COST OPTIMIZATION CHECKLIST                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ARCHITECTURE (40% potential savings)                                   │
│  □ Identify steps that don't need LLM (use regex, rules, templates)     │
│  □ Combine multiple LLM calls into one where possible                   │
│  □ Implement tiered processing (rules → cheap LLM → expensive LLM)      │
│  □ Limit self-reflection loops (max 2-3 iterations)                     │
│                                                                         │
│  CACHING (30% potential savings)                                        │
│  □ Enable LangChain caching for development                             │
│  □ Use long system prompts to benefit from OpenAI prefix caching        │
│  □ Cache embeddings for RAG applications                                │
│                                                                         │
│  PROMPTS (20% potential savings)                                        │
│  □ Remove unnecessary words and pleasantries                            │
│  □ Use JSON/structured output instead of prose                          │
│  □ Limit few-shot examples (1-2 instead of 5)                           │
│  □ Set max_tokens to prevent runaway outputs                            │
│  □ Use abbreviations for field names                                    │
│                                                                         │
│  MODEL SELECTION (10-50% potential savings)                             │
│  □ Audit each LLM call - does it need GPT-4o?                           │
│  □ Implement model routing based on task complexity                     │
│  □ Use GPT-4o-mini for classification, extraction, scoring              │
│  □ Reserve GPT-4o for reasoning, analysis, creative tasks               │
│                                                                         │
│  MONITORING (prevent cost surprises)                                    │
│  □ Set up Langfuse/observability for cost tracking                      │
│  □ Create daily/weekly cost alerts                                      │
│  □ Track cost per feature using tags                                    │
│  □ Review cost anomalies weekly                                         │
│                                                                         │
│  ADVANCED                                                               │
│  □ Use Batch API for non-urgent bulk processing (50% off)               │
│  □ Consider fine-tuning to replace long prompts                         │
│  □ Evaluate open-source models for specific tasks                       │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Summary: Key Takeaways

### Observability Tools Comparison

| Criteria | LangSmith | Langfuse |
|----------|-----------|----------|
| **Best for** | LangChain native | Framework-agnostic |
| **Deployment** | Cloud only | Cloud + self-hosted |
| **Cost tracking** | Limited | Built-in |
| **Open source** | No | Yes |
| **Integration** | Automatic | Callback handler |

### Langfuse Quick Reference

```python
# 1. Install
pip install langfuse

# 2. Configure
export LANGFUSE_SECRET_KEY="sk-lf-..."
export LANGFUSE_PUBLIC_KEY="pk-lf-..."

# 3. Use with LangGraph
from langfuse.callback import CallbackHandler
handler = CallbackHandler(user_id="...", session_id="...", tags=[...])
graph.invoke(state, config={"callbacks": [handler]})
```

### Cost Optimization Summary

| Strategy | Savings | Effort |
|----------|---------|--------|
| Architecture (fewer LLM calls) | 40% | High |
| Caching (semantic + exact) | 30% | Medium |
| Prompt optimization | 20% | Low |
| Model selection | 10% | Low |

---

## Next: Module 3 - Security & Guardrails

In the next module, we'll cover:
- Security concepts for LLM applications
- Prompt injection defenses
- Output validation techniques
- Guardrails frameworks overview

In [ ]:
# Module Complete!

print("""
╔═══════════════════════════════════════════════════════════════════════╗
║                                                                       ║
║  🎉 MODULE 2 COMPLETE: Observability & Cost Optimization             ║
║                                                                       ║
╠═══════════════════════════════════════════════════════════════════════╣
║                                                                       ║
║  ✅ What You Learned:                                                ║
║     • LangSmith vs Langfuse comparison                               ║
║     • Langfuse setup and integration with LangGraph                  ║
║     • Custom spans with @observe decorator                           ║
║     • Cost optimization strategies (architecture, caching, prompts)  ║
║                                                                       ║
║  📚 Resources:                                                        ║
║     • Langfuse: https://langfuse.com/docs                            ║
║     • LangSmith: https://docs.smith.langchain.com                    ║
║     • OpenAI Pricing: https://openai.com/pricing                     ║
║                                                                       ║
║  🛠️  Hands-on:                                                        ║
║     • Langfuse integration added to incident-postmortem project      ║
║     • See: session-2/projects/incident-postmortem/main.py            ║
║                                                                       ║
║  ➡️  Next: Module 3 - Security & Guardrails                          ║
║                                                                       ║
╚═══════════════════════════════════════════════════════════════════════╝
""")